In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from tika import parser
from bs4.element import Comment
from bs4 import BeautifulSoup

In [2]:
with open('JAKIM _ i-FIQH 2.0.html') as fopen:
    soup = BeautifulSoup(fopen.read())

In [7]:
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href', '').endswith('pdf')]
len(a)

14

In [8]:
!mkdir fiqh

In [14]:
for no, url in enumerate(a):
    filename = os.path.join('fiqh', os.path.split(url)[1])
    r = requests.get(url, verify = False)
    with open(filename, 'wb') as fopen:
        fopen.write(r.content)

/home/husein/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'i-fiqh.islam.gov.my'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/husein/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'i-fiqh.islam.gov.my'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/husein/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'i-fiqh.islam.gov.my'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/husein/.local/lib/python3

In [15]:
TIKA_HOST = 'http://localhost:9998'

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()


def parse_tika(file, minlen=2):
    raw_xml = parser.from_file(file, TIKA_HOST, xmlContent=True)
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            if len(t_) > minlen:
                t.append(t_)
        except Exception as e:
            pass
    
    return t

In [18]:
from glob import glob

files = glob('fiqh/*')

In [19]:
with open('i-fiqh.jsonl', 'w') as fopen:
    for f in tqdm(files):
        t = '\n'.join(parse_tika(f))
        data = {
            'file': f,
            'text': t,
        }
        fopen.write(f'{json.dumps(data)}\n')

100%|███████████████████████████████████████████| 14/14 [00:00<00:00, 47.81it/s]
